In [1]:
import requests
import json
import time

def create_pushshift_url(subreddit, from_ts, to_ts, size):
    url = 'https://api.pushshift.io/reddit/search/submission/?&size='+str(size)+'&after='+str(from_ts)+'&before='+str(to_ts)+'&subreddit='+str(subreddit)+'&fields=title,selftext,id,created_utc'
    return url

def get_data(subreddit, from_date, to_date, size, text_length):
    from_ts = int(time.mktime(from_date.timetuple()))
    to_ts = int(time.mktime(to_date.timetuple()))
    data = list()
    while len(data) < size:
        url = create_pushshift_url('CasualConversation', from_ts, to_ts, 500)
        r = requests.get(url)
        data_part = json.loads(r.text)['data']
        data_part = [item for item in data_part if 'selftext' in item and 'created_utc' in item and len(item['selftext']) < text_length and item['selftext'] != '[removed]' and item['selftext'] != '[deleted]']
        data_part = list(map(lambda item: {'id': item['id'], 'text': ' '.join([''.join(item['title'].split()), ' '.join(item['selftext'].split())]), 'time': item['created_utc']}, data_part))
        data = data + data_part
        from_ts = data[-1]['time'] + 1
        print("got {} items ({} total), next date: {}".format(len(data_part), len(data), from_ts))
    return data


In [2]:
from datetime import date
from_date = date.fromisoformat('2019-01-01')
to_date = date.fromisoformat('2020-01-01')

data = get_data("CasualConversation", from_date, to_date, 1000, 200)


got 50 items (50 total), next date: 1546419258
got 69 items (119 total), next date: 1546574457
got 65 items (184 total), next date: 1546773144
got 49 items (233 total), next date: 1546923649
got 81 items (314 total), next date: 1547091901
got 53 items (367 total), next date: 1547242244
got 65 items (432 total), next date: 1547397503
got 56 items (488 total), next date: 1547532787
got 40 items (528 total), next date: 1547695402
got 60 items (588 total), next date: 1547861570
got 54 items (642 total), next date: 1548028534
got 59 items (701 total), next date: 1548183605
got 53 items (754 total), next date: 1548343432
got 56 items (810 total), next date: 1548507326
got 43 items (853 total), next date: 1548697956
got 59 items (912 total), next date: 1548875549
got 66 items (978 total), next date: 1549049091
got 52 items (1030 total), next date: 1549214854


In [3]:
id = 0
for submission in data:
    id += 1
    print("processing " + str(id))
    url = 'https://api.pushshift.io/reddit/comment/search/' + '?link_id=' + submission['id'] + '&size=5' +  '&nest_level=1' + '&fields=id,body'
    r = requests.get(url)
    comments = json.loads(r.text)['data']
    submission['comments'] = [' '.join(c['body'].split()) for c in comments]

processing 1
processing 2
processing 3
processing 4
processing 5
processing 6
processing 7
processing 8
processing 9
processing 10
processing 11
processing 12
processing 13
processing 14
processing 15
processing 16
processing 17
processing 18
processing 19
processing 20
processing 21
processing 22
processing 23
processing 24
processing 25
processing 26
processing 27
processing 28
processing 29
processing 30
processing 31
processing 32
processing 33
processing 34
processing 35
processing 36
processing 37
processing 38
processing 39
processing 40
processing 41
processing 42
processing 43
processing 44
processing 45
processing 46
processing 47
processing 48
processing 49
processing 50
processing 51
processing 52
processing 53
processing 54
processing 55
processing 56
processing 57
processing 58
processing 59
processing 60
processing 61
processing 62
processing 63
processing 64
processing 65
processing 66
processing 67
processing 68
processing 69
processing 70
processing 71
processing 72
p

In [4]:
import csv
def create_csv(data, filename):
    upload_count = 0
    file = filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["ID","Message","Comment"]
        a.writerow(headers)
        for sub in data:
            if 'comments' not in sub:
                continue
            for c in sub['comments']:
                a.writerow([sub['id'], sub['text'], c])

In [5]:
create_csv(data, 'test_reddit2.csv')